# Over the Hills

Example histograms with Formula 1 data from
https://ergast.com/mrd/db

In [1]:
%autosave 0

Autosave disabled


In [2]:
from etl import DATADIR, ErgastF1
from viz import Plot

In [3]:
f1 = ErgastF1(DATADIR / 'ergast/f1.zip')
plot = Plot()